# Python 101
## Part XI.
---

## Time series

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
try:
    import seaborn as sns
except:
    pass

In [ ]:
BASE = './data/'

---
## Act I: Read historical USD-GBP-EUR currency rate data

Read data from the xlsx file.

In [ ]:
df = pd.read_excel(io=BASE+'currency_rates.xlsx', sheet_name='data')
df.head()

Set `Date` column as index (also sort the column).

In [ ]:
df = df.set_index('Date').sort_index()

#### Get the basic statistics about the data.

- Minimum

In [ ]:
df.min()

- Maximum

In [ ]:
df.max()

- Mean

In [ ]:
df.mean()

- Get every descriptive stat at once

In [ ]:
df.describe()

#### Let's focus on closing values!

In [ ]:
currencies = ['EUR/USD', 'GBP/USD', 'EUR/GBP']
def curr_list(suffix):
    return [currency + suffix for currency in currencies]

In [ ]:
df[curr_list(' Close')].plot()

In [ ]:
sns.boxplot(data=df[curr_list(' Close')])

Create a new column called 'diff' which is the difference between the opening and the closing prices. Plot it!

In [ ]:
for currency in currencies:
    df[currency + '_diff'] = df[currency +' High'] - df[currency + ' Low'] 

df[curr_list('_diff')].plot()

Let's have a closer look at GBP at June! (Brexit consequencies)

In [ ]:
df['2016-06-20':'2016-07-01'][curr_list(' Close')].plot()

In [ ]:
sns.boxplot(df['2016-06':'2016-07'][curr_list('_diff')])

In [ ]:
sns.boxplot(df['2016-06':'2016-07'][curr_list(' Close')])

---
## Act II: Advanced operations

Let's focus on GBP, and save the closing price to a separate `Series`!

In [ ]:
# [c]loseing [p]rice [f]rame
cpf = df['EUR/GBP Close']

### Subact: Moving statistics

The most common operation is the moving average (rolling mean). Feel free to experiment with the window size!

In [ ]:
mavg = cpf.rolling(window=40).mean()
mavg.tail()

We plot the last 5 because:

In [ ]:
mavg.head()

Why?

---

After the answer, plot the moving average! 

In [ ]:
mavg.plot()

Put the two plot into the same axis system.

In [ ]:
cpf.plot(label='Close')
mavg.plot(label='Close AVG')
plt.legend()

We can apply several other types of functions with moving window, the complete list can be found [here](http://pandas.pydata.org/pandas-docs/stable/computation.html#window-functions).  

---

### Subact: Shifted computations

How can we compute something based on yesterday's data?  
Use the `shift` function!  
Let's compute the difference between the days:
$${d_t} = {p_t} - {p_{t-1}}$$

In [ ]:
df['gbp_diff'] = cpf - cpf.shift(1)
df['gbp_diff'].head()

In [ ]:
df['gbp_diff'].describe()

In [ ]:
df['gbp_diff'].plot()

In [ ]:
df[df['gbp_diff'] > 0]['EUR/GBP Close'].describe()

---
## Act III: Resampling

You can resample data to different time frequency. Two main parameters for the `resample` method is the time period you resemple to and the method that you use. By default the method is mean. The list time frames is accessable from [here](http://pandas.pydata.org/pandas-docs/dev/timeseries.html#offset-aliases).   

In [ ]:
# Monthly mean
cpf.resample('M').mean().plot()

In [ ]:
# Weekly median
cpf.resample('W').median().plot()

In [ ]:
# 3 weekly minimum
cpf.resample('3W').min().plot()

In [ ]:
# quarter year max
cpf.resample('Q').max().plot()

---

## Let's do some...

<img align="left" width=150 src="pics/magic.gif">
<br style="clear:left;"/>

### Act IV: Cool library of the week: <a href="https://docs.python.org/2/library/collections.html">python's collections lib</a>

#### High-performance container datatypes
There are five awesome data types implemented in this library. My two favourites are:
- Counter

In [ ]:
from collections import Counter

data = ("Please, listen to me. The archbishop Lazarus, "
        "he led us down here to find the lost prince. "
        "The bastard led us into a trap! Now everyone is dead... "
        "Killed by a demon he called The Butcher. Avenge us! "
        "Find this butcher and slay him so that "
        "our souls may finally rest...").lower().split()

wordcount = Counter(data)
wordcount.most_common(10)

- defaultdict

In [ ]:
import string
from collections import defaultdict

words = defaultdict(set)

for word in data:
    words[word[0]].add(word)

words

---
## Final Act: your play time!
## It's your turn - write the missing code snippets!

#### 1. Plot the same metrics about the USD! Focus on the time of election!

#### 2. Which currency "won" the most on the election of Trump?

#### 3. Compute the moving sum on the "groving" days (when the currency of the moved up)!  

#### 4.a. Compute the moving standard deviation function
$${sd} = \sqrt{{E}[({x}-\mu)^2]}$$
on the dataset with 60 window size! 

#### 4.b. Plot it! 
Plot the resampled (quarterly, mean) dataframe into the same coordinate system as well (use the `style='--g'` argument in the `plot` method)!

In [ ]:
def msd(x):
    return np.sqrt(np.square(x - x.mean()).mean())